# Depth Anything Mono → SBS Converter (Colab)

This notebook converts a standard **mono video** into **SBS (Side‑By‑Side) stereo** for VR headsets (e.g., Pico 4) using **Depth Anything V2** depth estimation and reprojection.

## Workflow (recommended)
1. **Clone the repo** and install dependencies + models**
2. *(Optional)* **Run a short smoke-test**
3. **Create one or more jobs** (URLs or files) → generates `/content/Depth-Anything-collab-notebook/jobs/*.yaml`
4. **Run the job runner** to process all pending jobs
5. **Export finished videos to Google Drive**

> Tip: Colab sessions reset. You typically need to run **Clone + Install** again each session.


## 1) Quick Start (one‑click setup)

If you are in a hurry, run the next cell once. It clones the repo and runs the Colab installer.
After that, jump to **“3) Generate YAML jobs”** to create conversion jobs, then run **“4) Run jobs”**.


In [1]:
#@title Quick Start: clone + install
!rm -rf /content/Depth-Anything-collab-notebook
!git clone https://github.com/hashtag1138/Depth-Anything-collab-notebook.git
%cd /content/Depth-Anything-collab-notebook
!python install_collab.py --with-widgets --smoke-test --model all


Cloning into 'Depth-Anything-collab-notebook'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 72 (delta 28), reused 54 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 227.99 KiB | 1.61 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/Depth-Anything-collab-notebook

🧾 Environment info
 - timestamp: 2026-02-15T21:39:21.669951+00:00
 - python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
 - repo: /content/Depth-Anything-collab-notebook
 - ffmpeg: /usr/bin/ffmpeg
 - nvidia-smi: /opt/bin/nvidia-smi

🟦 RUN: bash -lc nvidia-smi -L || true
GPU 0: Tesla T4 (UUID: GPU-444690d7-e2af-1776-85f5-7f488501491f)
✅ ffmpeg found.
✅ widgets already installed.
✅ Prepared dirs:
 - /content/jobs
 - /content/work
 - /content/Depth-Anything-collab-notebook/checkpoints
🟨 Cloning Depth-Anything-V2 into: /content/Depth-Anything-collab-notebook/Depth-Anything-V2

🟦 RUN: bash -

## Suggested presets (recipes)

These are practical starting points. You can tune them in the **job generator**.

### Fast preview (to validate depth / parallax quickly)
- `mode=preview`
- `preview_interval`: **30–120** (higher = faster)
- `encoder`: **vits** (fast)
- `input_size`: **384–448**
- `batch`: increase until stable (GPU memory limits)
- `max_shift`: **12–24**, `alpha`: **0.85–0.95**

### “Pico 4 2K” output (common comfortable target)
- `sbs_w=2560`, `sbs_h=1440` (or keep `sbs_h=0` if your autosize converter handles it)

### Higher quality (slower)
- `encoder`: **vitb** or **vitl**
- `input_size`: **518** or **640**
- `batch`: reduce if you hit OOM
- Consider `quality mode=nvenc` when available

> Fun fact: “SBS 2560×1440” is often called “2K” in VR pipelines because each eye effectively gets ~1280×1440.


### 2) Run install tests (with required args)

This cell performs a quick validation test of the installation.
It generates a calibration image and runs a short preview conversion.
If it succeeds, your environment is ready.


In [ ]:
%cd /content/Depth-Anything-collab-notebook

# 1) Choisir un converter existant dans le repo
CONVERTER = "./mono_to_sbs_pico4_v2_autosize.py"  # adapte si ton fichier a un autre nom

# 2) Trouver / créer une image de calibration
# Option A: si tu as déjà une image dans le repo (ex: assets/calibration.png), mets son chemin ici.
# Option B (par défaut): on en génère une simple avec ffmpeg (pas besoin de pillow).
IMAGE = "/content/calibration_3840x2160.png"

# Génération d'une mire simple si absente
!test -f "$IMAGE" || ffmpeg -y -f lavfi -i "testsrc2=size=3840x2160:rate=1" -frames:v 1 "$IMAGE" >/dev/null 2>&1

# 3) Lancer le test avec les args requis
!python test_install.py \
  --image "$IMAGE" \
  --converter "$CONVERTER" \
  --depth_repo "./Depth-Anything-V2" \
  --outdir "_test_install" \
  --duration 6 \
  --fps 30 \
  --sbs_w 2560 \
  --encoder vitb \
  --batch 1 \
  --input_size 518 \
  --max_shift 24 \
  --alpha 0.90 \
  --preview


### 3) Générer jobs/*.yaml (format EXACT run_job.py / new_job.py)

This cell opens a small UI to create conversion jobs.

You can:
- paste video URLs (YouTube, etc.)
- select local/Drive files
- choose preview vs full mode
- adjust depth + quality settings

Click **Generate YAML jobs** to write `jobs/*.yaml`, then run the job runner.


In [2]:
import re, time, uuid
from pathlib import Path
import yaml
import ipywidgets as W
from IPython.display import display, clear_output

from ipyfilechooser import FileChooser

REPO = Path.cwd()
JOBS_DIR = REPO / "jobs"
JOBS_DIR.mkdir(parents=True, exist_ok=True)

def slug(s: str, maxlen=80):
    s = (s or "").strip()
    s = re.sub(r"[^a-zA-Z0-9._-]+", "_", s)
    return s[:maxlen].strip("_") or "job"

# ---------- Sources ----------
urls_area = W.Textarea(
    value="",
    placeholder="1 URL par ligne (YouTube, etc.)",
    description="URLs",
    layout=W.Layout(width="100%", height="140px"),
)

default_fc_dir = "/content/drive/MyDrive" if Path("/content/drive/MyDrive").exists() else str(REPO)
fc = FileChooser(default_fc_dir)
fc.use_dir_icons = True
fc.title = "Choisis un fichier (puis clique Add)"
add_file_btn = W.Button(description="Add file", button_style="info", icon="plus")
clear_files_btn = W.Button(description="Clear files", button_style="", icon="trash")
files_box = W.Select(options=tuple(), description="Files", layout=W.Layout(width="100%", height="120px"))

def add_file(_):
    p = fc.selected
    if not p:
        return
    cur = list(files_box.options)
    if p not in cur:
        cur.append(p)
        files_box.options = tuple(cur)

def clear_files(_):
    files_box.options = tuple()

add_file_btn.on_click(add_file)
clear_files_btn.on_click(clear_files)

# ---------- Runtime / paths ----------
converter_path = W.Text(value="mono_to_sbs_pico4_v2_autosize.py", description="converter")
depth_repo     = W.Text(value="Depth-Anything-V2", description="depth_repo")
checkpoints_dir= W.Text(value="checkpoints", description="ckpt_dir")
log_dir        = W.Text(value="logs", description="log_dir")

download_dir   = W.Text(value="work/downloads", description="dl_dir")
work_download_format = W.Text(value="best", description="ytdlp_fmt")

# ---------- Convert params ----------
mode = W.Dropdown(options=["full","preview"], value="full", description="mode")
preview_interval = W.IntText(value=30, description="preview_int")

sbs_w = W.IntText(value=2560, description="sbs_w")
sbs_h = W.IntText(value=0, description="sbs_h (0=null)")
encoder = W.Dropdown(options=["vits","vitb","vitl"], value="vits", description="encoder")
batch = W.IntSlider(value=8, min=1, max=128, step=1, description="batch", continuous_update=False)
input_size = W.Dropdown(options=[384, 448, 512, 518, 640], value=518, description="input_size")
max_shift = W.IntSlider(value=24, min=0, max=64, step=1, description="max_shift", continuous_update=False)
alpha = W.FloatSlider(value=0.90, min=0.0, max=1.0, step=0.01, description="alpha", continuous_update=False)

# ---------- Quality ----------
q_mode = W.Dropdown(options=["auto","nvenc","x264"], value="auto", description="q_mode")
cq = W.IntSlider(value=24, min=14, max=40, step=1, description="cq", continuous_update=False)
nv_preset = W.Dropdown(options=["p1","p2","p3","p4","p5","p6","p7"], value="p5", description="nv_preset")
video_codec = W.Dropdown(options=["auto","h264","hevc"], value="auto", description="codec")
crf = W.IntSlider(value=28, min=16, max=40, step=1, description="crf", continuous_update=False)
x264_preset = W.Dropdown(options=["ultrafast","superfast","veryfast","faster","fast","medium","slow","slower","veryslow"], value="fast", description="x264_preset")

# ---------- Output ----------
out_dir = W.Text(value="outputs", description="out_dir")
name_mode = W.Dropdown(options=["auto","prefix"], value="auto", description="name_mode")
name_prefix = W.Text(value="sbs", description="prefix")

job_name_prefix = W.Text(value="job", description="job_prefix")
overwrite_yaml = W.Checkbox(value=False, description="overwrite YAML")

out = W.Output()

def parse_urls(text: str):
    urls = []
    for line in (text or "").splitlines():
        u = line.strip()
        if u and not u.startswith("#"):
            urls.append(u)
    return urls

def make_job_struct(name: str):
    return {"job": {"name": name}}

def build_runtime():
    return {
        "converter_path": converter_path.value.strip(),
        "depth_repo": depth_repo.value.strip(),
        "checkpoints_dir": checkpoints_dir.value.strip(),
        "log_dir": log_dir.value.strip(),
    }

def build_convert():
    return {
        "mode": mode.value,
        "preview_interval": int(preview_interval.value),
        "sbs_w": int(sbs_w.value),
        "sbs_h": (None if int(sbs_h.value) == 0 else int(sbs_h.value)),
        "encoder": encoder.value,
        "batch": int(batch.value),
        "input_size": int(input_size.value),
        "max_shift": int(max_shift.value),
        "alpha": float(alpha.value),
    }

def build_quality():
    return {
        "mode": q_mode.value,
        "cq": int(cq.value),
        "nv_preset": nv_preset.value,
        "video_codec": video_codec.value,
        "crf": int(crf.value),
        "x264_preset": x264_preset.value,
    }

def build_output():
    return {
        "dir": out_dir.value.strip(),
        "name_mode": name_mode.value,
        "prefix": name_prefix.value.strip() if name_mode.value == "prefix" else None,
    }

def write_yaml(doc: dict, name_hint: str):
    ts = time.strftime("%Y%m%d_%H%M%S")
    uid = uuid.uuid4().hex[:8]
    fname = f"{slug(job_name_prefix.value)}_{ts}_{uid}_{slug(name_hint)}.yaml"
    path = JOBS_DIR / fname
    if path.exists() and not overwrite_yaml.value:
        raise FileExistsError(f"{path} existe déjà. Coche overwrite YAML pour écraser.")
    with open(path, "w", encoding="utf-8") as f:
        yaml.safe_dump(doc, f, sort_keys=False, allow_unicode=True)
    return path

gen_btn = W.Button(description="Generate YAML jobs", button_style="success", icon="cogs")
list_btn = W.Button(description="List jobs/", icon="list")

def on_generate(_):
    with out:
        clear_output()

        urls = parse_urls(urls_area.value)
        files = list(files_box.options)

        if not urls and not files:
            print("⚠️ Aucun URL ni fichier.")
            return

        runtime = build_runtime()
        convert = build_convert()
        quality = build_quality()
        output = build_output()

        created = []

        # URLs → ytdlp
        for u in urls:
            hint = (u.split("/")[-1] or "url")[:60]
            jobname = f"job_{time.strftime('%Y%m%d_%H%M%S')}"
            doc = make_job_struct(jobname)
            doc["source"] = {
                "kind": "ytdlp",
                "url": u,
                "download_dir": download_dir.value.strip(),
                "format": work_download_format.value.strip() or "best",
            }
            doc["output"] = output
            doc["runtime"] = runtime
            doc["convert"] = convert
            doc["quality"] = quality
            created.append(write_yaml(doc, hint))

        # Files → file
        for p in files:
            pth = Path(p)
            hint = pth.stem or pth.name
            jobname = f"job_{time.strftime('%Y%m%d_%H%M%S')}"
            doc = make_job_struct(jobname)
            doc["source"] = {
                "kind": "file",
                "path": str(pth),
            }
            doc["output"] = output
            doc["runtime"] = runtime
            doc["convert"] = convert
            doc["quality"] = quality
            created.append(write_yaml(doc, hint))

        print(f"✅ {len(created)} job(s) écrits dans {JOBS_DIR}")
        for p in created:
            print(" -", p.name)

def on_list(_):
    with out:
        clear_output()
        jobs = sorted(JOBS_DIR.glob("*.yaml"))
        print(f"📁 {JOBS_DIR} — {len(jobs)} YAML")
        for p in jobs[:200]:
            print(" -", p.name)

gen_btn.on_click(on_generate)
list_btn.on_click(on_list)

left = W.VBox([
    W.HTML("<b>Sources</b>"),
    urls_area,
    W.HTML("<b>File picker</b>"),
    fc,
    W.HBox([add_file_btn, clear_files_btn]),
    files_box,
])

right = W.VBox([
    W.HTML("<b>runtime</b>"),
    converter_path,
    W.HBox([depth_repo, checkpoints_dir]),
    log_dir,
    W.HTML("<b>source (ytdlp)</b>"),
    W.HBox([download_dir, work_download_format]),
    W.HTML("<b>convert</b>"),
    W.HBox([mode, preview_interval]),
    W.HBox([sbs_w, sbs_h]),
    W.HBox([encoder, input_size]),
    W.HBox([batch, max_shift]),
    alpha,
    W.HTML("<b>quality</b>"),
    W.HBox([q_mode, cq, nv_preset]),
    W.HBox([video_codec, crf, x264_preset]),
    W.HTML("<b>output</b>"),
    W.HBox([out_dir, name_mode, name_prefix]),
    W.HBox([job_name_prefix, overwrite_yaml]),
    W.HBox([gen_btn, list_btn]),
])

display(W.HBox([left, right], layout=W.Layout(gap="20px")))
display(out)


Output()

### 4) Run the jobs

This cell starts processing all pending jobs.
It reads the YAML files under `jobs/` and performs the conversions.

Progress is printed in the output. Keep this cell running until done.


In [ ]:
!python -u run_job.py

### 5) Exporter les vidéos finies vers Google Drive (choix du dossier)

This cell exports finished videos to Google Drive.

Steps:
1) Pick a destination folder on Drive
2) Click **Scan videos**
3) Click **Send to Drive** (rsync shows progress and is recommended for large files)


In [ ]:
import os, shutil, subprocess, sys
from pathlib import Path
import ipywidgets as W
from IPython.display import display, clear_output

# 1) Monter Drive si pas déjà monté
if not Path("/content/drive/MyDrive").exists():
    try:
        from google.colab import drive
        drive.mount("/content/drive")
    except Exception as e:
        raise RuntimeError("Impossible de monter Google Drive dans cette session.") from e

from ipyfilechooser import FileChooser

REPO = Path.cwd()

# Dossiers où chercher des vidéos "finies"
SEARCH_DIRS = [
    REPO / "outputs",
    # optionnel : REPO / "work",
]

VIDEO_EXTS = {".mp4", ".mkv", ".mov", ".webm"}

def find_videos():
    vids = []
    for d in SEARCH_DIRS:
        if not d.exists():
            continue
        for p in d.rglob("*"):
            if p.is_file() and p.suffix.lower() in VIDEO_EXTS:
                vids.append(p)
    # tri stable
    vids.sort(key=lambda p: (p.suffix.lower(), p.stat().st_mtime))
    return vids

# FileChooser en mode dossier
fc = FileChooser("/content/drive/MyDrive")
fc.use_dir_icons = True
fc.title = "Choisis un dossier de destination sur Drive"
fc.show_only_dirs = True  # important: répertoires uniquement

copy_mode = W.Dropdown(
    options=[("rsync (recommandé)", "rsync"), ("copy (shutil)", "copy")],
    value="rsync",
    description="mode",
)

dry_run = W.Checkbox(value=False, description="dry_run")
btn_scan = W.Button(description="Scan videos", icon="search")
btn_send = W.Button(description="Send to Drive", button_style="success", icon="upload")

out = W.Output()

def on_scan(_):
    with out:
        clear_output()
        vids = find_videos()
        print("📁 Recherche dans:")
        for d in SEARCH_DIRS:
            print(" -", d)
        print(f"\n🎞️ Vidéos trouvées: {len(vids)}")
        for p in vids[:200]:
            print(" -", p.relative_to(REPO) if p.is_relative_to(REPO) else p)

def on_send(_):
    with out:
        clear_output()
        dest = fc.selected_path or fc.selected
        if not dest:
            print("⚠️ Choisis un dossier Drive dans le picker.")
            return
        dest = Path(dest)
        if not dest.exists():
            print("⚠️ Dossier destination introuvable:", dest)
            return

        vids = find_videos()
        if not vids:
            print("⚠️ Aucune vidéo à envoyer.")
            return

        print("📦 Destination Drive:", dest)
        print(f"🎞️ {len(vids)} vidéo(s) à copier.\n")

        # On copie en conservant l'arborescence relative sous outputs/
        for src in vids:
            # base dir = outputs ou autre; on essaie de garder un chemin relatif utile
            rel = None
            for base in SEARCH_DIRS:
                if base.exists():
                    try:
                        rel = src.relative_to(base)
                        rel = Path(base.name) / rel  # prefix "outputs/..."
                        break
                    except ValueError:
                        pass
            if rel is None:
                rel = src.name

            dst = dest / rel
            dst.parent.mkdir(parents=True, exist_ok=True)

            if dry_run.value:
                print("DRY:", src, "->", dst)
                continue

            if copy_mode.value == "rsync":
                # -a: conserve timestamps, -h: human, --progress: progression
                cmd = ["bash", "-lc", f'rsync -ah --info=progress2 "{src}" "{dst}"']
                subprocess.run(cmd, check=False)
            else:
                shutil.copy2(src, dst)
                print("copied:", dst)

        print("\n✅ Export terminé.")

btn_scan.on_click(on_scan)
btn_send.on_click(on_send)

display(W.VBox([
    fc,
    W.HBox([copy_mode, dry_run, btn_scan, btn_send], layout=W.Layout(gap="10px")),
    out
]))
